# Datast 2 : Digit Recognizer

In this dataset we will explore how our plumber optimizer performs on multiclass classification task with a relatively small dataset

### Importing the libraries 

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
import time
import psutil
import os
import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score

In [2]:
import optuna
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score, make_scorer
import time
import numpy as np
import matplotlib.pyplot as plt
import logging
"""
Random Forest Hyperparameter Optimization

Description:
    This file implements a technique called the 'Plumber Optimizer', which provides automated tuning for RandomForest models.
    The core idea divides the tuning process into two phases: a general and exploratory phase, followed by a second, more focused phase.
    It also implements an early stopping technique if a significant number of trials pass with no improvement.
    The optimizer allows the user to choose between F1 and accuracy as the metric score for optimization in classification tasks,
    while using MSE for regression tasks.

Usage:
    By providing the training data, specifying whether it's a classification or a regression task, and choosing if F1 should be used
    instead of accuracy, the Plumber Optimizer will automatically suggest a number of trials that balance performance and efficiency.
    The tuning process begins using the Optuna library. In addition to fully automated tuning, a visualization function is available
    that provides diagrams to help users visualize what happened behind the scenes.

Authors:
    Mohammed Aldahmani
    Ali Al-Ali
    Abdalla Alzaabi

Date:
    November 14, 2024

Dependencies:
    - optuna
    - numpy
    - pandas
    - sklearn
    - matplotlib
    - psutil
"""

class PlumberOptimizer:

    def __init__(self, X, y, classification=True, f1=False, verbose=0):
        """
        Constructor for PlumberOptimizer class.
        Args:
            X: Training data.
            y: Target vector.
            classification (bool): Set to True if the task is a classification task, False for regression.
            f1 (bool): Set to True to use F1 score as the evaluation metric for classification tasks.
            verbose (int): If set to 0, suppresses logging; otherwise, Optuna will print the results of each trial.
        
        Initializes the class by training a single RandomForest model to measure the time it takes, then automatically calculates
        a suitable number of trials based on this single model's training time and prints the result to the user.
        """
        self.X = X
        self.y = y
        self.best_param = None
        self.importance = None
        self.study = None
        self.improved_study = None
        self.best_value = float('-inf') 
        self.trials_since_last_improvement = 0
        self.classification = classification
        self.f1 = f1
        self.num_trials = self.compute_num_trials()
        print(f"Suggested number of trails is {self.num_trials} ")
        if verbose==0:
            optuna.logging.set_verbosity(optuna.logging.WARNING)




    def compute_single_model_time(self):
        """
        Helper function to measure the duration to train a single RandomForest model.
        Returns:
            float: Time in seconds it takes to train the model.
        """
        model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
        start_time = time.time()
        model.fit(self.X, self.y)
        return time.time() - start_time
    


    
    def compute_num_trials(self):
        """
        Helper function that uses a logistic function to compute the number of trials.
        Models that take less than a second to train will have a large number of trials (150),
        while models that take more than 30 seconds will have fewer trials (about 30).
        Returns:
            int: Computed number of trials.
        """
        single_model_time = self.compute_single_model_time()
        print("A single model takes ",single_model_time,"seconds to run")


        return int(30 + (220) / (1 + np.exp(0.2 * (single_model_time - 1))))
    


    def objective(self, trial):
        """
        An Optuna Objective function, this is the phase 1 function that tunes all 5 parameters each having a wide
        range of values. 
        """

        params = {
            'n_estimators': trial.suggest_int('n_estimators', 50, 300, step=50),
            'max_depth': trial.suggest_int('max_depth', 10, 30, step = 5),
            'min_samples_split': trial.suggest_int('min_samples_split', 2, 15),
            'min_samples_leaf': trial.suggest_int('min_samples_leaf', 1, 15),
            'max_features': trial.suggest_categorical('max_features', ['sqrt', 'log2'])
        }
        if self.classification:
            model = RandomForestClassifier(**params, n_jobs=-1, random_state=42)
            if not self.f1:
                score = cross_val_score(model, self.X, self.y, cv=3, n_jobs=-1, scoring='accuracy').mean()
            else:
                scorer = make_scorer(f1_score, average='weighted')
                score = cross_val_score(model, self.X, self.y, cv=3, n_jobs=-1, scoring=scorer).mean()
        else:
            model = RandomForestRegressor(**params, n_jobs=-1, random_state=42)
            score = cross_val_score(model, self.X, self.y, cv=3, n_jobs=-1, scoring='neg_mean_squared_error').mean()

        return score




    def focused_objective(self, trial):
        """
        A second Optuna Objective function, this is the phase 2 function that tunes only the top 3 parm according to their 
        importance value determained by Optuna. It then assign a narower set of values for each parameter
        """

        # Sort parameter importances and select the top three
        top_params = dict(sorted(self.importance.items(), key=lambda item: item[1], reverse=True)[:3])

        params = {}
        for param in ['n_estimators', 'max_depth', 'min_samples_split', 'min_samples_leaf', 'max_features']:
            if param in top_params:
                base_value = self.best_param[param]
                # Adjust the range slightly based on the base_value
                if param == 'n_estimators':  # Specific handling for n_estimators
                    params[param] = trial.suggest_int(param, max(1, base_value - 50), base_value + 50, step=25)
                elif param == 'min_samples_split':  # Specific handling for n_estimators
                    params[param] = trial.suggest_int(param, max(2, base_value - 10), base_value + 10)
                elif isinstance(base_value, int):
                    params[param] = trial.suggest_int(param, max(1, base_value - 5), base_value + 5)
                else:  # for categorical parameters like max_features
                    params[param] = trial.suggest_categorical(param, ['sqrt', 'log2'])
            else:
                params[param] = self.best_param[param]

        if self.classification:
            model = RandomForestClassifier(**params, n_jobs=-1, random_state=42)
            if not self.f1:
                score = cross_val_score(model, self.X, self.y, cv=3, n_jobs=-1, scoring='accuracy').mean()
            else:
                scorer = make_scorer(f1_score, average='weighted')
                score = cross_val_score(model, self.X, self.y, cv=3, n_jobs=-1, scoring=scorer).mean()
        else:
            model = RandomForestRegressor(**params, n_jobs=-1, random_state=42)
            score = cross_val_score(model, self.X, self.y, cv=3, n_jobs=-1, scoring='neg_mean_squared_error').mean()

        return score
    


    def early_stopping_callback(self, study, trial):
        """
        An optuna callback function, in our case it is used to stop early if no improvamant is made after 0.4*num_trials steps since the
        last best score is found or 30 for cases where 0.4*num_trials < 30
        """
        threshold = max(self.num_trials * 0.40, 30)
        if study.best_value == self.best_value:
            self.trials_since_last_improvement += 1
        else:
            self.best_value = study.best_value
            self.trials_since_last_improvement = 0

        if self.trials_since_last_improvement >= threshold:
            study.stop()  





    def optimize(self):
        """
        The main function that is to be called by the user, it begins phase1, obtain the importance, then start the improved stude
        phase 2. finaly it prints the best paramaters alongside the best score, 
        returns the best params dictionary
        Returns:
            dic: Best value for each parameter
        """

        ### Phase 1: self.study tunse all 5 paramaters, in a broad set of values

        self.study = optuna.create_study(direction='maximize')
        self.study.optimize(self.objective, self.num_trials, callbacks=[self.early_stopping_callback])
        self.best_param = self.study.best_params                                       # Obtain the best Params
        self.importance = optuna.importance.get_param_importances(self.study)          # Obtain the Importance of each param
        
        
        ### Phase 2: using only the top 3 parameters, begin a focused study
        self.best_value = float('-inf')                                             
        self.trials_since_last_improvement = 0
        self.improved_study = optuna.create_study(direction='maximize')
        self.improved_study.optimize(self.focused_objective, int(self.num_trials/2), callbacks=[self.early_stopping_callback])

        # Update best parameters with improvements from phase 2 if the best score from phase 2 is higher than phase 1 best score
        if self.improved_study.best_value > self.study.best_value:
            self.best_param.update(self.improved_study.best_params)


        print("Best Parameters:", self.best_param)
        print("Best Score:", self.improved_study.best_value)
        return  self.best_param
    



    def visualize(self):
        """
        A function that is to be called if the user wish to visualize phase 1 params importance (figure 1)
        as well as pararral coordinate of the optimization process for phase 2.
        Returns:
            tuple: Figures representing parameter importances and parallel coordinates of the optimization process.
        """
        print("fig1: After the first phase, This is how important each variable is")
        fig1 = optuna.visualization.plot_param_importances(self.study)
        print("")
        print("fig2: Visualizing the focused study plot parallel coordinate")
        fig2 = optuna.visualization.plot_parallel_coordinate(self.improved_study)
        return fig1, fig2


### Loading the Data and performing feature engineering:

In [3]:


data = pd.read_csv('/kaggle/input/digitrec/train.csv')

X = data.drop('label', axis=1)
y = data['label']

X_normalized = X / 255.0
pca = PCA(n_components=0.90)  
X_reduced = pca.fit_transform(X_normalized)


X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

### Testing the performance and efficiency of the PlumberOptimizer

In [4]:
plumber_time = np.zeros(3,'float')
plumber_mem = np.zeros(3,'float')
plumber_cpu =  np.zeros(3,'float')
plumber_score = np.zeros(3,'float')
for i in range(3):
    opt = PlumberOptimizer(X_train, y_train)

    process = psutil.Process(os.getpid())
    start_time = time.time()
    start_cpu = process.cpu_percent(interval=None)
    start_mem = process.memory_info().rss / (1024 * 1024) 


    best = opt.optimize()

    end_time = time.time()
    end_cpu = process.cpu_percent(interval=None)
    end_mem = process.memory_info().rss / (1024 * 1024)  


    plumber_time[i] = end_time - start_time
    plumber_mem[i] = end_mem - start_mem
    plumber_cpu[i] = end_cpu - start_cpu

    rf = RandomForestClassifier(**best , random_state=42 )
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_test)


    plumber_score[i] = accuracy_score(y_test, y_pred)


print("\n" *3)
print("*************")
print(f"Execution Time: mean={np.mean(plumber_time):.2f} seconds, std={np.std(plumber_time)} seconds, max={np.max(plumber_time)} seconds, min={np.min(plumber_time)} seconds")
print(f"CPU Usage: mean={np.mean(plumber_cpu):.2f}%, std={np.std(plumber_cpu):.2f}%, max={np.max(plumber_cpu):.2f}%, min={np.min(plumber_cpu):.2f}%")
print(f"Memory Usage: mean={np.mean(plumber_mem):.2f} MB, std={np.std(plumber_mem):.2f} MB, max={np.max(plumber_mem):.2f} MB, min={np.min(plumber_mem):.2f} MB")
print(f"Validation Score: mean={np.mean(plumber_score)} , std={np.std(plumber_score)} , max={np.max(plumber_score)} , min={np.min(plumber_score)} ")

A single model takes  25.327974319458008 seconds to run
Suggested number of trails is 31 
Best Parameters: {'n_estimators': 300, 'max_depth': 23, 'min_samples_split': 4, 'min_samples_leaf': 1, 'max_features': 'log2'}
Best Score: 0.9454761904761906
A single model takes  25.271666765213013 seconds to run
Suggested number of trails is 31 
Best Parameters: {'n_estimators': 300, 'max_depth': 25, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt'}
Best Score: 0.9423809523809523
A single model takes  25.12181830406189 seconds to run
Suggested number of trails is 31 
Best Parameters: {'n_estimators': 275, 'max_depth': 35, 'min_samples_split': 6, 'min_samples_leaf': 1, 'max_features': 'sqrt'}
Best Score: 0.9438690476190477




*************
Execution Time: mean=2152.74 seconds, std=268.1613689403954 seconds, max=2432.2526710033417 seconds, min=1791.015058040619 seconds
CPU Usage: mean=0.40%, std=0.00%, max=0.40%, min=0.40%
Memory Usage: mean=6.12 MB, std=7.94 MB, max=17.35 MB

In [5]:
# fig1, fig2 = opt.visualize()

# fig1.show()
# fig2.show()

### Testing the performance and efficiency of the GridSearch

In [6]:
param_grid = {
    'n_estimators': [200, 400],  
    'max_features': ['sqrt', 'log2'], 
    'max_depth': [ 10, 30, 50], 
    'min_samples_split': [2, 8, 14],  
    'min_samples_leaf': [2,  8,  14]  
}

rf = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=3, n_jobs=-1, scoring='accuracy')

process = psutil.Process(os.getpid())
start_time = time.time()
start_cpu = process.cpu_percent(interval=None)
start_mem = process.memory_info().rss / (1024 * 1024)  

grid_search.fit(X_train, y_train)

end_time = time.time()
end_cpu = process.cpu_percent(interval=None)
end_mem = process.memory_info().rss / (1024 * 1024)  

print(f"Execution Time: {end_time - start_time} seconds")
print(f"CPU Usage: {end_cpu - start_cpu}%")
print(f"Memory Usage: {end_mem - start_mem} MB")


rf = RandomForestClassifier(** grid_search.best_params_, random_state=42 )
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print(f"Validation Accuracy: {accuracy}")



/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Execution Time: 5667.087579488754 seconds
CPU Usage: 2.0%
Memory Usage: 97.40625 MB
Validation Accuracy: 0.9477380952380953


### Testing the performance and efficiency of the RandomSearch

In [7]:
param_dist = {
    'n_estimators': np.arange(50, 500),
    'max_features': ['sqrt', 'log2'],
    'max_depth': np.arange(10, 50),
    'min_samples_split': np.arange(2, 30),
    'min_samples_leaf': np.arange(1, 30)
}
rand_time = np.zeros(3,'float')
rand_mem = np.zeros(3,'float')
rand_cpu =  np.zeros(3,'float')
rand_score = np.zeros(3,'float')
for i in range(3):
    rf = RandomForestClassifier(random_state=42)
    random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_dist,
                                   n_iter=45, cv=3, n_jobs=-1)
    process = psutil.Process(os.getpid())
    start_time = time.time()
    start_cpu = process.cpu_percent(interval=None)
    start_mem = process.memory_info().rss / (1024 * 1024)  

    random_search.fit(X_train, y_train)

    end_time = time.time()
    end_cpu = process.cpu_percent(interval=None)
    end_mem = process.memory_info().rss / (1024 * 1024)  


    rand_time[i] = end_time - start_time
    rand_mem[i] = end_mem - start_mem
    rand_cpu[i] = end_cpu - start_cpu

    rf = RandomForestClassifier(** random_search.best_params_ , random_state=42 )
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)


    rand_score[i] = accuracy_score(y_test, y_pred)


print(f"Execution Time: mean={np.mean(rand_time):.2f} seconds, std={np.std(rand_time)} seconds, max={np.max(rand_time)} seconds, min={np.min(rand_time)} seconds")
print(f"CPU Usage: mean={np.mean(rand_cpu):.2f}%, std={np.std(rand_cpu):.2f}%, max={np.max(rand_cpu):.2f}%, min={np.min(rand_cpu):.2f}%")
print(f"Memory Usage: mean={np.mean(rand_mem):.2f} MB, std={np.std(rand_mem):.2f} MB, max={np.max(rand_mem):.2f} MB, min={np.min(rand_mem):.2f} MB")
print(f"Validation Score: mean={np.mean(rand_score)} , std={np.std(rand_score)} , max={np.max(rand_score)} , min={np.min(rand_score)} ")

Execution Time: mean=2159.84 seconds, std=73.73932292993157 seconds, max=2258.1127712726593 seconds, min=2080.481866121292 seconds
CPU Usage: mean=2.90%, std=0.65%, max=3.50%, min=2.00%
Memory Usage: mean=2.88 MB, std=1.50 MB, max=4.98 MB, min=1.53 MB
Validation Score: mean=0.9446428571428571 , std=0.0013712025827913369 , max=0.9461904761904761 , min=0.9428571428571428 


### Testing the performance and efficiency of the BayesSearchCV

In [8]:
param_dist = {
    'n_estimators': Integer(50, 500),
    'max_features': Categorical(['sqrt', 'log2']),
    'max_depth': Integer(10, 50),
    'min_samples_split': Integer(2, 30),
    'min_samples_leaf': Integer(1, 30)
}

rand_time = np.zeros(3,'float')
rand_mem = np.zeros(3,'float')
rand_cpu =  np.zeros(3,'float')
rand_score = np.zeros(3,'float')
for i in range(3):
    rf = RandomForestClassifier(random_state=42)
    bayes_search = BayesSearchCV(estimator=rf, search_spaces=param_dist,
                                 n_iter=40, cv=3, n_jobs=-1)
    process = psutil.Process(os.getpid())
    start_time = time.time()
    start_cpu = process.cpu_percent(interval=None)
    start_mem = process.memory_info().rss / (1024 * 1024) 

    bayes_search.fit(X_train, y_train)
    
    end_time = time.time()
    end_cpu = process.cpu_percent(interval=None)
    end_mem = process.memory_info().rss / (1024 * 1024)  

    rand_time[i] = end_time - start_time
    rand_mem[i] = end_mem - start_mem
    rand_cpu[i] = end_cpu - start_cpu

    rf = RandomForestClassifier(** bayes_search.best_params_ , random_state=42 )
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)
    rand_score[i] = accuracy_score(y_test, y_pred)

print(f"Execution Time: mean={np.mean(rand_time):.2f} seconds, std={np.std(rand_time)} seconds, max={np.max(rand_time)} seconds, min={np.min(rand_time)} seconds")
print(f"CPU Usage: mean={np.mean(rand_cpu):.2f}%, std={np.std(rand_cpu):.2f}%, max={np.max(rand_cpu):.2f}%, min={np.min(rand_cpu):.2f}%")
print(f"Memory Usage: mean={np.mean(rand_mem):.2f} MB, std={np.std(rand_mem):.2f} MB, max={np.max(rand_mem):.2f} MB, min={np.min(rand_mem):.2f} MB")
print(f"Validation Score: mean={np.mean(rand_score)} , std={np.std(rand_score)} , max={np.max(rand_score)} , min={np.min(rand_score)} ")

Execution Time: mean=2996.16 seconds, std=140.05438087838198 seconds, max=3157.4255545139313 seconds, min=2815.938412666321 seconds
CPU Usage: mean=10.57%, std=0.59%, max=11.40%, min=10.10%
Memory Usage: mean=421.02 MB, std=10.20 MB, max=435.16 MB, min=411.49 MB
Validation Score: mean=0.9496825396825397 , std=0.00020234204419021528 , max=0.9498809523809524 , min=0.9494047619047619 
